In [14]:
from pathlib import Path
import vectorbt as vbt
from hydra.SuperSim import load_prices

def load_portfolio(name):    
    output_dir = Path.cwd().joinpath('..', 'output')
    filepath = output_dir.joinpath(name)
    return vbt.Portfolio.load(filepath)

vbt.settings.ohlcv["column_names"] = {
    "open": "open",
    "high": "high",
    "low": "low",
    "close": "close",
    "volume": "volume",
}

pair='XBTUSD'
start='2018-05-15 00:00:00'
end='2018-05-15 1:30:00'
interval=1

#portfolio = load_portfolio(f"{pair} {start} {end} {interval} Aroon 100-101.portfolio")
prices = {}
prices[1] = load_prices(pair, '../data/kraken', start, end, 1)
prices[5] = load_prices(pair, '../data/kraken', start, end, 5)
prices[15] = load_prices(pair, '../data/kraken', start, end, 15)
prices[60] = load_prices(pair, '../data/kraken', start, end, 60)
prices[720] = load_prices(pair, '../data/kraken', start, end,720)
prices[1440] = load_prices(pair, '../data/kraken', start, end, 1440)
AROONOSC = vbt.IndicatorFactory.from_talib("AROONOSC")
# printd(help(AROONOSC.run))
aroonosc = AROONOSC.run(
    prices[1]["high"], prices[1]["low"], timeperiod=[100]
)



In [15]:
import plotly.graph_objects as go
import pandas as pd
import hydra.utils as utils
from datetime import datetime, timedelta


candlestick = go.Candlestick(
        x=prices[1440].index,
        open=prices[1440]["open"],
        high=prices[1440]["high"],
        low=prices[1440]["low"],
        close=prices[1440]["close"],
        hoverinfo="all",
    )
print(prices[1440])

def parsedatetime(dt):
    return pd.to_datetime(dt)
    # return datetime.strptime(dt, '%Y-%m-%d %H:%M')
start='2018-05-15 00:30:00'
end='2018-05-15 1:00:00'

start_date = parsedatetime(start)
end_date = parsedatetime(end)
delta = end_date - start_date
delta_m = delta.total_seconds() / 60
if delta_m < 2000:
    interval = 1
elif delta_m < 2000 * 5:
    interval = 5
elif delta_m < 2000 * 15:
    interval = 15
elif delta_m < 2000 * 60:
    interval = 60
elif delta_m < 2000 * 720:
    interval = 720
else:
    interval = 1440
precise_price = prices[interval].loc[start_date:end_date]
orig_price = prices[1440]
the_price = pd.concat([orig_price, precise_price]).sort_index()
f = open('log.txt', "a")
fakestart = start_date - timedelta(minutes=10)
fakeend = end_date + timedelta(minutes=10)
f.write(f"[{utils.now()}] {start} {end} {delta} {delta_m} {interval} \n{precise_price} \n {fakestart} {fakeend} \n{the_price.loc[fakestart:fakeend]}\n")
f.close()


              open    high     low   close       volume
time                                                   
2018-05-15  8661.6  8862.0  8421.0  8462.8  3875.260105
